In [1]:
import neptune.new as neptune

run = neptune.init(project='arsde/J22TAO')

https://app.neptune.ai/arsde/J22TAO/e/JTAO-36


Info (NVML): RM has detected an NVML/RM version mismatch.. GPU usage metrics may not be reported. For more information, see https://docs.neptune.ai/you-should-know/what-can-you-log-and-display#hardware-consumption


Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


In [2]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings("ignore")

In [3]:
train_mode = True
features_selection = True

In [4]:
PARAMS = {
    "FC_cut": 0.65,
    "N": int(1.15e6),
    "random_state": 22,
    "max_depth": 9,
    "early_stopping_rounds": 10,
    "n_folds": 10,    
}

run['PARAMS'] = PARAMS

In [5]:
path = "/mnt/cephfs/ml_data/TAO_detsim_J22/"
data = pd.read_csv(f'{path}processed_data/ProcessedTrain/ProcessedTrain.csv.gz')

In [6]:
from sklearn.utils import shuffle

FC_cut = PARAMS['FC_cut']
N = PARAMS['N']
random_state = PARAMS['random_state']
max_depth = PARAMS['max_depth']
early_stopping_rounds = PARAMS['early_stopping_rounds']
n_folds = PARAMS['n_folds']

data = shuffle(data, random_state=random_state)
data = data.reset_index(drop=True)
data = data[data['edepR'] < FC_cut]
data = data[:N]
data.head()

,AccumCharge,nPMTs,R_cc,rho_cc,x_cc,y_cc,z_cc,gamma_z_cc,gamma_y_cc,gamma_x_cc,...,pe_75p,pe_80p,pe_85p,pe_90p,pe_95p,edep,edepX,edepY,edepZ,edepR
0,35482.0,3999.0,0.365263,0.364290,0.338427,0.134812,-0.026646,-0.073146,0.397121,2.462711,...,10.0,12.0,15.0,18.0,27.0,8.520335,0.468565,0.200681,-0.033771,0.510849
1,26449.0,3881.0,0.436751,0.369734,0.056378,-0.365410,-0.232483,-0.628785,-1.527498,0.130175,...,7.0,9.0,11.0,15.0,23.0,6.394049,0.079346,-0.524534,-0.343030,0.631744
3,6304.0,2817.0,0.328890,0.312968,-0.125881,-0.286536,-0.101091,-0.323008,-1.774786,-0.414293,...,3.0,3.0,4.0,4.0,6.0,1.668662,-0.164486,-0.408590,-0.155663,0.467154
9,13865.0,3606.0,0.347917,0.191859,0.088948,-0.169994,-0.290236,-1.512758,-0.560001,0.264448,...,5.0,5.0,7.0,8.0,11.0,3.526859,0.123386,-0.231287,-0.426100,0.500279
10,40302.0,4007.0,0.368905,0.364372,0.192190,0.309564,-0.057652,-0.158224,1.542799,0.610346,...,12.0,14.0,17.0,22.0,30.0,9.680871,0.268719,0.425487,-0.085745,0.510491


In [7]:
from sklearn.metrics import mean_squared_error
from neptune.new.types import File
import plotly.express as px
import plotly.io as pio
pio.templates.default = 'plotly_white'

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def plot_results(df, all_features_metrics):
    
    all_features_metric = all_features_metrics[0]
    eps = all_features_metrics[1]
    all_features_metrics_lower = all_features_metric - eps
    all_features_metrics_upper = all_features_metric + eps
    
    df['color'] = 1
    
    print(df)
    
    fig = px.scatter(
        df,
        x="Added feature",
        y="MAPE, %",
        error_y='current_metric_stds',
        color='color',
        animation_frame="Number of features",
    )
        
    fig.add_hline(
        y=all_features_metric,
        line=dict(dash='dash')
    )

    fig.add_hrect(
        y0=all_features_metrics_lower, 
        y1=all_features_metrics_upper,
        fillcolor="darkred",
        line_width=1,
        opacity=0.25,
    )
    
    fig.update_layout(
        xaxis = dict(
            showline=True,
            ticks='outside',
            mirror=True,
            linecolor='black',
            showgrid=True,
            gridcolor='grey',
            gridwidth=0.25,
        ),

        yaxis = dict(
            showline=True,
            ticks='outside',
            mirror=True,
            linecolor='black',
            tick0=0,
            showgrid=True,
            gridcolor='grey',
            gridwidth=0.25,
            zeroline=True,
            zerolinecolor='black',
            zerolinewidth=0.25
        ),

        font=dict(
            family="Times New Roman",
            size=16,
            color="Black"
        )
    )

    run["plot_results_anim"].upload(File.as_html(fig))

In [8]:
size = int(1e6)
n_feats = data.shape[1] - 5

X_val = data.iloc[size:, :-5]
y_val = data.iloc[size:, -5]
data = data[:size]

In [9]:
from xgboost import XGBRegressor
from sklearn.model_selection import KFold

if train_mode:    
    mape_scores = []
    kfold = KFold(n_folds, shuffle=True, random_state=random_state)   
    
    for train, test in tqdm(kfold.split(np.array(data)), "Folds... ", leave=False):        
        xgbreg = XGBRegressor(
                max_depth=max_depth,
                learning_rate=0.08,
                n_estimators=10000,
                tree_method='gpu_hist',
                random_state=random_state
        )
                            
        X_train = data.iloc[train, :n_feats]
        y_train = data.iloc[train, n_feats]
                            
        X_test = data.iloc[test, :n_feats]
        y_test = data.iloc[test, n_feats]

        xgbreg.fit(X_train, y_train,
                   verbose=False,
                   eval_set=[(X_val, y_val)],
                   early_stopping_rounds=early_stopping_rounds)
        
        y_predict = xgbreg.predict(X_test)
        mape = mean_absolute_percentage_error(y_test, y_predict)
        mape_scores.append(mape)
    
    result = np.array([np.mean(mape_scores), np.std(mape_scores)])
    np.savez_compressed('feature_selection/all_features_metrics.npz', a=result)

Folds... : 0it [00:00, ?it/s]

In [10]:
all_features_metrics = np.load('feature_selection/all_features_metrics.npz', allow_pickle=True)['a']
run['all_features_metrics'] = all_features_metrics
print(all_features_metrics)

[0.88738198 0.00561805]


In [ ]:
if features_selection:
    all_features_metric = np.load('feature_selection/all_features_metrics.npz', allow_pickle=True)['a'][0]
    eps = np.load('feature_selection/all_features_metrics.npz', allow_pickle=True)['a'][1]

    opt_features = []
    current_metrics = []
    current_metric_stds = []
    current_metric = 100
    current_metric_std = 0

    features = data.iloc[:, :-5].columns
    features = features.drop(opt_features)
    
    df_plot = pd.DataFrame()

    while True:#abs(all_features_metric - current_metric) > current_metric_std:
        metrics = []
        metric_stds = []
        
        for feature in tqdm(features, "Features loop"):
            mape_scores = []
            kfold = KFold(n_folds, shuffle=True, random_state=random_state)   
            
            for train, test in tqdm(kfold.split(np.array(data)), "Folds... ", leave=False):        
                X = data.iloc[:, :-5][opt_features+[feature]]
                y = data.iloc[:, -5]

                xgbreg = XGBRegressor(
                    max_depth=max_depth,
                    learning_rate=0.08,
                    n_estimators=10000,
                    random_state=random_state,
                    tree_method='gpu_hist',
                )

                X_train = X.iloc[train, :]
                y_train = y.iloc[train]

                X_test = X.iloc[test, :]
                y_test = y.iloc[test]

                xgbreg.fit(X_train, y_train,
                           verbose=False,
                           eval_set=[(X_val[opt_features+[feature]], y_val)],
                           early_stopping_rounds=early_stopping_rounds)

                y_predict = xgbreg.predict(X_test)
                mape = mean_absolute_percentage_error(y_test, y_predict)
                mape_scores.append(mape)
            
            metric = np.mean(mape_scores)
            metric_std = np.std(mape_scores)
            metrics.append(metric)
            metric_stds.append(metric_std)
        
        best_metric_ind = np.argmin(metrics)
        current_metric = metrics[best_metric_ind]
        current_metrics.append(current_metric)

        current_metric_std = metric_stds[best_metric_ind]
        current_metric_stds.append(current_metric_std)

        opt_features.append(features[best_metric_ind])
        features = features.drop(features[best_metric_ind])

        print(current_metrics)
        print(current_metric_stds)
        print(opt_features)

        np.savez_compressed('feature_selection/opt_features.npz', a=np.array(opt_features))
        np.savez_compressed('feature_selection/current_metrics.npz', a=np.array(current_metrics))
        np.savez_compressed('feature_selection/current_metric_stds.npz', a=np.array(current_metric_stds))
        
        run['opt_features'] = opt_features
        run['current_metrics'] = current_metrics
        run['current_metric_stds'] = current_metric_stds

        df = pd.DataFrame([opt_features]).T
        df.columns = ['Added feature']
        df['MAPE, %'] = current_metrics
        df['current_metric_stds'] = current_metric_stds
        df['Number of features'] = len(opt_features)
        df_plot = df_plot.append(df)
        
        plot_results(df_plot, all_features_metrics)      

Features loop:   0%|          | 0/91 [00:00<?, ?it/s]

Folds... : 0it [00:00, ?it/s]

Folds... : 0it [00:00, ?it/s]

Folds... : 0it [00:00, ?it/s]

Folds... : 0it [00:00, ?it/s]

...

Folds... : 0it [00:00, ?it/s]

Folds... : 0it [00:00, ?it/s]

Folds... : 0it [00:00, ?it/s]

Folds... : 0it [00:00, ?it/s]

Folds... : 0it [00:00, ?it/s]

In [ ]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

current_metrics = np.load('feature_selection/current_metrics.npz', allow_pickle=True)['a']
opt_features = np.load('feature_selection/opt_features.npz', allow_pickle=True)['a']
current_metric_stds = np.load('feature_selection/current_metric_stds.npz', allow_pickle=True)['a']
all_features_metric = np.load('feature_selection/all_features_metrics.npz', allow_pickle=True)['a'][0]
eps = np.load('feature_selection/all_features_metrics.npz', allow_pickle=True)['a'][1]

all_features_metrics_lower = all_features_metric - eps
all_features_metrics_upper = all_features_metric + eps

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=opt_features,
        y=current_metrics,
        error_y=dict(
            type='data',
            width=10,
            array=current_metric_stds
        ),
        marker=dict(
            color='black'
        )
    )
)

fig.add_hline(
    y=all_features_metric,
    line=dict(dash='dash')
)

fig.add_hrect(
    y0=all_features_metrics_lower, 
    y1=all_features_metrics_upper,
    fillcolor="darkred",
    line_width=1,
    opacity=0.25,
)

fig.update_yaxes(title='MAPE, %')
fig.update_xaxes(title='Added feature')

fig.update_layout(

    xaxis = dict(
        showline=True,
        ticks='outside',
        mirror=True,
        linecolor='black',
        showgrid=True,
        gridcolor='grey',
        gridwidth=0.25,
    ),

    yaxis = dict(
        showline=True,
        ticks='outside',
        mirror=True,
        linecolor='black',
        tick0=0,
        showgrid=True,
        gridcolor='grey',
        gridwidth=0.25,
        zeroline=True,
        zerolinecolor='black',
        zerolinewidth=0.25
    ),
    
    font=dict(
        family="Times New Roman",
        size=16,
        color="Black"
    )
)

fig.show()
run["plot_results"].upload(File.as_html(fig))